In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename):     
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    if any([ds.Modality != 'DX', ds.PatientPosition not in ['PA','AP'], ds.BodyPartExamined != 'CHEST']):
        print(f'The image {filename} is invalid. Model only supports  PA/AP positions of Chest X-Rays')
        return
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size):
#     proc_img = img / 255.0
    proc_img = (img - img_mean) / img_std
    proc_img = resize(proc_img, img_size, anti_aliasing=True)
    return proc_img


# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    with open(model_path, 'r') as json_file:
        savedModel = json_file.read()
    
    model = keras.models.model_from_json(savedModel)
    model.load_weights(weight_path)
    return model


# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    result = model.predict(img)
    pred = result[0]
    if pred >= thresh:
        prediction = 'Pneumonia'
    else:
        prediction = 'No Pneumonia'
    
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json' #path to saved model
weight_path = 'xray_class_my_model.best.hdf5' #path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
# img_mean = # loads the mean image value they used during training preprocessing
# img_std = # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.0806327760219574 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_mean, img_std = np.mean(img), np.std(img)
    img_proc = preprocess_image(img, img_mean, img_std, IMG_SIZE)
    pred = predict_image(my_model, img_proc,thresh)
    print(pred)

Load file test1.dcm ...
Pneumonia
Load file test2.dcm ...
Pneumonia
Load file test3.dcm ...
Pneumonia
Load file test4.dcm ...
The image test4.dcm is invalid. Model only supports  PA/AP positions of Chest X-Rays
Load file test5.dcm ...
The image test5.dcm is invalid. Model only supports  PA/AP positions of Chest X-Rays
Load file test6.dcm ...
The image test6.dcm is invalid. Model only supports  PA/AP positions of Chest X-Rays
